## Work flow for single cell tracing analysis

1. Convert annotations to um units
2. (V3d) Resample in 1um steps
3. Downsample to 25ums, will also generate indices for endings
4. (Console) Transform points according to how the sample was aligned to template (run Transform_points.py under elastix directory)
5. Identify points in the corresponding atlas
6. Make tiff stack with points only, in full z sections (functional for sample imaged in coronal plane, 

WIP for samples in horizontal consider that both AP and DV planes were cut...)


### update august 2021 for anatomy MS

to generate tiff stacks for axons and dendrite as well as region with counts in excel files

In [1]:
import os
import tkinter.filedialog as fdialog
import Neuron_analysis as na
from Neuron_analysis import *
import numpy as np

import pandas as pd
import SimpleITK as sitk


In [3]:
na.points.convert_anno()
# repeat for both axons and dendrites

Converting pixels to ums....
Done!


#### Now, go to v3d and resample these files in 1um steps
Under vaa3d plugin- resample. Repeat for both axons and dendrites

Then come back and proceed to next step

In [3]:
na.points.downsample_anno()
# repeat for both axons and dendrites

Resampled annotation step size is 1 um in x y z. downsampling to 25 um. dowmsample ratio is 25.0.


Press Enter to continue... 


AL157D_25voxel_trace_1umStepsize.txt
Downsampling annotation done.
Finding endings of the annotations...
There are 81 endings
D:/AL157/AL157D_endings.csv
File saved in the output directory.


#### Now, go to console and transform these points
use Transform_points.py

In [24]:
in_dir = fdialog.askdirectory(title='Please select the input directory')
# ie: D:\2PT electroporation time\AL066
axon= fdialog.askopenfilename(title='Selecte the transformed points for axons')
dendrite= fdialog.askopenfilename(title='Selecte the transformed points for dendrites')

In [38]:
# files= os.listdir(in_dir)
# ending_files= [os.path.join(in_dir,i) for i in files if '_endings' in i]
# axon_ending= [i for i in ending_files if 'D_' not in i][0]
# dendrite_ending=[i for i in ending_files if 'D_' in i][0]


In [25]:
out_dir='D:/Complete_points/'
mouse_name=na.find_mousename(in_dir)
out_name=out_dir+mouse_name

### old section

mostly for sample 2ara files

In [6]:
points,atlas_name=na.get_pt_natlas(file_type,in_dir,full=False)
# reads in the points and get the corresponding atlas(cropped version cus we need to extract the cropped plane from its name)
na.find_crop(atlas_name)
# extract the cropped sections ie. section 100-394
points_full=na.points.refill_section(points,atlas_name)
# fills in the missing section by adding the leading slice to the z coordinates of the downsampled points
na.atlas.make_tif(points_full, na.fullatlas_name,out_dir+mouse_name, is_axon)
# projects the downsampled points(now in full z plane) to a tiff file, now ready to merge with the complete atlas and template

Starting to saving tif files..


C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:/Complete_points/AL175_axons.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


In [13]:
region_counts, points_in_atlas=na.points.make_pd(points_full,file_ending,out_name,is_axon)
# find the corresponding atlas ID for each point, store in pd datafile with counts for main and ends
# also outputs an excel file
na.points.make_point_csv(points_full, points_in_atlas,out_name,is_axon)
# outputs a csv file with downsampled points and their associated id

In [14]:
na.points.check_points(points_in_atlas)
# check for IDs that are without atlas label

In [15]:
points_original= fdialog.askopenfilename(title='Selecte the original resampled points')
# asks for the non-downsampled annotation, usually the eswc file that has been converted to um and resampled at 1 um steps
# typically in petersen server analysis folder

original_withID=na.points.findID_origional(points_original, points_in_atlas,out_name,is_axon)
#makes a csv file with original points and their associated id


In [89]:
na.whatis()

what is it that you look for? 484682528


Atlas id= 529    484682528
Name: id, dtype: int64
Name= 529    commissural branch of stria terminalis
Name: safe_name, dtype: object
Hemisphere= 529    3
Name: hemisphere_id, dtype: int64


In [23]:
points,atlas_name=na.get_pt_natlas(file_type,in_dir,full=False)


In [26]:
na.find_crop(atlas_name)


TypeError: 'NoneType' object is not subscriptable

## update aug 2021 for anatomy MS
## for ara2sample 

In [26]:
##useful functions

def check_points(points_in_atlas):
    '''Checks whether all your points' ID is within the atlas labels
    Input: matching ID of the points (this is the second output from na.make_pd)
    '''
    id_inatlas=[]
    for x in atlas_labels['region_id']:
        intID = int(x)
        id_inatlas.append(intID)

    # need to format this first ourselves,otherwise problematic for 0 and very large numbers (idk why)    

    num_of_zeros = [i for i, x in enumerate(points_in_atlas) if x == 0]
    # find the indices for which carries an id =0
    
    unique_id=set(points_in_atlas)
    
    for id_inbrain in unique_id:
        if id_inbrain not in id_inatlas:
            if id_inbrain==0:
                print(f'There are {len(num_of_zeros)} points with ID= {id_inbrain}, this index is outside of the brain, consider possible suboptimal image registration')
            else: 
                print(id_inbrain,'this index does not exist in allen reference atlas, see https://github.com/ChristophKirst/ClearMap/issues/37')
            warnings.warn('Some points do not have corresponding labels')
    return 

def make_pd(all_points, out_name, atlas_name, axon=1):
    ''' 
    For ara2sample only
    Takes in all points (as a list, usually the output of na.refill_section or na.get_pt_natlas ) and formulates a pd structure.
    Input: downsampled points (in transformix compatible format), name of corresponding indicies of endings 
    ouputs: a pandas dataframe with anatomical regions and their corresponding total points count and ending points count, list of atlas ID for each point
    
    '''
                        
    image= sitk.ReadImage(atlas_name)
    atlas =sitk.GetArrayFromImage(image)
        
    points_in_atlas=[int(atlas[i[2], i[1],i[0]]) for i in all_points ]
    #find an ID for all points

    unique_id=set(points_in_atlas)

    our_regions=atlas_labels.loc[atlas_labels['region_id'].isin (unique_id)]

    id_withcounts=[]
    for i in unique_id:
        id_withcounts.append([i, points_in_atlas.count(i)])

    new_df= pd.DataFrame(id_withcounts, columns=['region_id', 'Total_counts'])
    our_regionWcounts=pd.merge(our_regions, new_df)
    
    if axon==1:
        out_name=out_name + 'axons_region_with_counts.xls'
    else:
        out_name=out_name + 'dendrites_region_with_counts.xls'
    
    our_regionWcounts.to_excel(out_name,index=None,header=True)

    return our_regionWcounts.sort_values(by=['Total_counts']), points_in_atlas

In [28]:
atlas_labels=atlas_labels=pd.read_csv('D:\Allenbrainatlas\ARA_25_micron_mhd_ccf2017\wraped\ccf_2017_itksnap_labels.csv')


In [29]:
is_axon=0

if is_axon==1:
    file_type=axon
elif is_axon==0:
    file_type=dendrite
# put 0 if working with dendrites

In [30]:
points,atlas_name=na.get_pt_natlas(file_type,in_dir)
# read points and get path name of the corresponding atlas

In [31]:
if is_axon :
    na.atlas.make_tif_1(points, atlas_name,out_dir+mouse_name, is_axon)
else:
    na.atlas.make_tif(points, atlas_name,out_dir+mouse_name, is_axon)
# save points as a tiff file for visualization and overlay
# for dendrites, keep the intensities different since...its too cramped up and will look bad!

Starting to saving tif files..


C:\Users\liu\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:/Complete_points/AL142_dendrites.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


In [32]:
points_in_atlas= na.points.find_point_id(points,atlas_name)
# find id of points in corresponding atlas

In [33]:
check_points(points_in_atlas)
#we should only see id=0 as possibility now since now is using the itksnap_wraped atlas

There are 239 points with ID= 0, this index is outside of the brain, consider possible suboptimal image registration


NameError: name 'warnings' is not defined

In [35]:
points_in_atlas= np.where(points_in_atlas==0, 844, points_in_atlas) 
# replace id= 0 with 844 (ssp-bfd layer1)

if is_axon :
    points_in_atlas= np.where(points_in_atlas==800 , 215 , points_in_atlas) 
    points_in_atlas= np.where(points_in_atlas==1092 , 215 , points_in_atlas)
    # replace id= 800 (subependymal zone) and 1092 (lateral ventricle) with 215(caudoputamen)
    # especially for AL131

check_points(points_in_atlas)
# check again to see if 0 disappears

In [36]:
1092 in points_in_atlas

False

In [37]:
region_counts=na.points.make_pd_ara2sample(points_in_atlas.tolist(),atlas_labels,out_name,is_axon)

## Optional

In [86]:
points_original= fdialog.askopenfilename(title='Selecte the original resampled points')
# asks for the non-downsampled annotation, usually the eswc file that has been converted to um and resampled at 1 um steps
# typically in petersen server analysis folder

original_withID=na.points.findID_origional(points_original, points_in_atlas,out_name,is_axon)
#makes a csv file with original points and their associated id

FileNotFoundError: [Errno 2] No such file or directory: ''

## stops here


In [42]:
atlas_labels

,region_id,acronym
0,1,6b
1,2,A13
2,3,AAA
3,4,ACA
4,5,ACA1
...,...,...
1322,1323,vsp
1323,1324,vtd
1324,1325,x
1325,1326,y


array([146, 146, 146, ..., 152, 152, 152])

In [58]:
def make_tif(all_points, atlas_name, outname,axon=1):
    ''' Project downsampled points on to a tiff stack, useful for overlaping with brain or template (ie, in imageJ)
    input: downsampled points in a list containing x y z ordinates as int, directory containing it (this is also the output directory) and whether annotation is axon or not (default True)
    example: [[12, 13, 25],
             [13, 14, 25],...]
    
    output: a tiff stack with the same dimensions of the brain/template/atlas mhd files with downsampled points only
    each point has a value of the number of occurences (since downsampling combines multiple points as one)
    '''
        
    print('Starting to saving tif files..')
    
    atlas= sitk.ReadImage(atlas_name)
    svolume=np.zeros(atlas.GetSize())
    #columns, rows, planes
    
    zplanes=[]
    for i in all_points:
        zplanes.append( i[2])
    zplanes=np.unique(zplanes)
    temp=np.zeros(atlas.GetSize()[0:2])
    thepoints=np.asarray(all_points)

    for i in zplanes:
        index= thepoints[:,2]==i
        uindex,counts=np.unique(thepoints[index],return_counts=True, axis=0)
        for lines in uindex:
            coord1,coord2=lines[0:2]
            temp[coord1][coord2]= 1
        svolume[:,:,i]=temp #write this in 
        temp=np.zeros(atlas.GetSize()[0:2]) #reset the empty plane after each z
        
    
    coronal_planetmp= np.swapaxes(np.int16(svolume),0,2)
    #for some reason, if just save stuff as tiff, it will save x planes of yz view
    #here we shift the 3rd dimension with the first dimension to obtain xy view
    if axon==1:
        out_name=outname + '_axons.tif'
    else:
        out_name=outname + '_dendrites.tif'

    io.imsave(out_name,coronal_planetmp)
    return 

Starting to saving tif files..


0